In [1]:
import pandas as pd
import numpy as np

In [2]:
import zipfile
import tensorflow as tf
import sys

In [3]:
import checkmate
from checkmate import BestCheckpointSaver

In [4]:
import matplotlib.pyplot as plt

In [5]:
seq_len = None

In [6]:
def remove_outliers(data, threshold=73):
    titles = {i: i.lower() for i in data.columns}
    return (data.rename(columns=titles).query('expected <= @threshold').copy())

In [7]:
def seq_pad(data, size, steps):
    arr = []
    for i in data:
        _ = i.copy()
        _.resize(steps, size)
        arr.append(_)
    arr = np.array(arr)
    return arr

# Data Loading

training - 20 days of each month
test - remaining days
Both sets are independent
Calendar time and location information omitted:
    impossible to construct a local validation holdout subset independent from the rest of the training set
    no way to ensure that any two gauge readings are not correlated in time/space

In [8]:
#zf1 = zipfile.ZipFile('../../train.zip')
#zf2 = zipfile.ZipFile('../../test.zip')
#train = pd.read_csv(zf1.open(zipfile.ZipFile.namelist(zf1)[0]))
#test = pd.read_csv(zf2.open(zipfile.ZipFile.namelist(zf2)[0]))
train = pd.read_csv('../../train.csv', nrows=60000).pipe(remove_outliers)
test = pd.read_csv('../../test.csv', nrows=50000)

In [9]:
train.head()

,id,minutes_past,radardist_km,ref,ref_5x5_10th,ref_5x5_50th,ref_5x5_90th,refcomposite,refcomposite_5x5_10th,refcomposite_5x5_50th,...,rhohv_5x5_90th,zdr,zdr_5x5_10th,zdr_5x5_50th,zdr_5x5_90th,kdp,kdp_5x5_10th,kdp_5x5_50th,kdp_5x5_90th,expected
0,1,3,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
1,1,16,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
2,1,25,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
3,1,35,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
4,1,45,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254


In [10]:
train_t = train.copy()

5-layer deep stacked Bi-RNN, 64-256 hidden units, dense after each hidden stack, 
last layer at each time position vector is fed into a dense layer with 1 output and ReLU. Take mean of the predictions from the entire top layer


# Preprocessing
exclude the rain gauges with readings above 70mm.
replace missing radar feature values with zero
each timestamp as a component in the feature vector, sequential nature of the input is preserved

In [11]:
#train_t['count_id'] = train_t.groupby('Id')['Id'].transform('count')
#train_t['count_None_Ref'] = train_t[~np.isnan(train_t.Ref)].transform('count')
train_nones = train_t[~np.isnan(train_t['ref'])]['id'].unique()
train_full = train_t['id']

In [12]:
train_cleaned = train_t[np.in1d(train_full, train_nones)].fillna(0.0).reset_index(drop=True)

In [13]:
ids = train_cleaned['id'].unique()
train_ids = np.random.choice(ids,
                               size=np.ceil(len(ids) * (9/10.)).astype(int),
                               replace=False)

In [14]:
train_data = train_t.loc[train_t['id'].isin(train_ids)]
val_data = train_t.loc[~train_t['id'].isin(train_ids)]
#[train.columns != 'expected']

In [15]:
train_data.head()

,id,minutes_past,radardist_km,ref,ref_5x5_10th,ref_5x5_50th,ref_5x5_90th,refcomposite,refcomposite_5x5_10th,refcomposite_5x5_50th,...,rhohv_5x5_90th,zdr,zdr_5x5_10th,zdr_5x5_50th,zdr_5x5_90th,kdp,kdp_5x5_10th,kdp_5x5_50th,kdp_5x5_90th,expected
6,2,1,2.0,9.0,5.0,7.5,10.5,15.0,10.5,16.5,...,0.998333,0.3750,-0.1250,0.3125,0.8750,1.059998,-1.410004,-0.350006,1.059998,1.016
7,2,6,2.0,26.5,22.5,25.5,31.5,26.5,26.5,28.5,...,1.005000,0.0625,-0.1875,0.2500,0.6875,NaN,NaN,NaN,1.409988,1.016
8,2,11,2.0,21.5,15.5,20.5,25.0,26.5,23.5,25.0,...,1.001667,0.3125,-0.0625,0.3125,0.6250,0.349991,NaN,-0.350006,1.759994,1.016
9,2,16,2.0,18.0,14.0,17.5,21.0,20.5,18.0,20.5,...,1.001667,0.2500,0.1250,0.3750,0.6875,0.349991,-1.059998,0.000000,1.059998,1.016
10,2,21,2.0,24.5,16.5,21.0,24.5,24.5,21.0,24.0,...,0.998333,0.2500,0.0625,0.1875,0.5625,-0.350006,-1.059998,-0.350006,1.759994,1.016


In [16]:
y_train = train_data.groupby('id')['expected'].last().values

y_val = val_data.groupby('id')['expected'].last().values

In [17]:
train_data = train_data.drop('expected', 1).fillna(0.0)
val_data = val_data.drop('expected', 1).fillna(0.0)

In [18]:
train_data_arr = train_data.groupby('id').apply(lambda x: x.values.astype('float')).values
val_data_arr = val_data.groupby('id').apply(lambda x: x.values.astype('float')).values

seq_lens_train = train_data.groupby('id').size().values
seq_lens_val = val_data.groupby('id').size().values

In [19]:
seq_lens_train[:10]

array([12, 13, 15, 12, 12,  7, 10, 10, 12, 12])

In [20]:
train_data_arr.shape, y_train.shape, val_data_arr.shape, y_val.shape 

((2837,), (2837,), (2360,), (2360,))

In [21]:
train_data_arr.shape[0], train_data_arr[0].shape[0], train_data_arr[0].shape[1]

(2837, 12, 23)

In [22]:
seq_len = max([seq_lens_train.max(), seq_lens_val.max()])#choose the length of the sequence
neurons = [64, 128, 256]
n_in = train_data_arr[0].shape[1]#length of the feature vector of each element in a sequence
n_out = 1#dim of the output feture vector

In [23]:
val_pad = seq_pad(val_data_arr, n_in, seq_len)

### Model

In [24]:
tf.reset_default_graph()

In [25]:
X = tf.placeholder(dtype=tf.float32, shape=[None, seq_len , n_in])
y = tf.placeholder(tf.float32, [None])
seq_length = tf.placeholder(dtype=tf.int16, shape=[None])

with tf.variable_scope('lstm-rnn', initializer=tf.contrib.layers.variance_scaling_initializer()):
    rnn_cell_1 = tf.contrib.rnn.LSTMCell(num_units=neurons[1])
    rnn_cell_2 = tf.contrib.rnn.LSTMCell(num_units=neurons[1])
    multi_layer_cell = tf.contrib.rnn.MultiRNNCell([rnn_cell_1, rnn_cell_2])
    outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32,
                                        sequence_length=seq_length)
    drop = tf.layers.dropout(inputs=states[1], rate=0.4)
    y_pred = tf.layers.dense(drop, n_out)

In [33]:
learning_rate = 1e-3

error = (y-y_pred)
loss = tf.reduce_mean(tf.square(error), name="loss")
global_step = tf.Variable(0, name='global_step', trainable=False)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
minimizer = optimizer.minimize(loss, global_step=global_step)
minimizer_val = optimizer.minimize(loss, global_step=global_step)

mae = tf.reduce_mean(tf.abs(error))

checkpoint_dir = './models/'
best_ckpt_saver = BestCheckpointSaver(save_dir=checkpoint_dir, num_to_keep=3, maximize=False)

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
n_samples = train_data_arr.shape[0]
n_epochs = 30
batch_size = len(train_data_arr) / 25
#printbatch_size
loss_hist = {'mae': {'train': [], 'val': []}, 'loss':{'train':[], 'val':[]}}

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        rand_idx = np.random.permutation(np.arange(n_samples))
        train_loss_list = []
        train_mae_list = []
        for batch_i in range(int(n_samples // batch_size)):
            batch_idx = rand_idx[int(batch_i*batch_size):int((batch_i+1)*batch_size)]
            X_batch = seq_pad(train_data_arr[batch_idx], n_in, seq_len)
            y_batch = y_train[batch_idx]
            len_batch = seq_lens_train[batch_idx]
            loss_train, _, step = sess.run(
                    [loss, minimizer, global_step],
                    feed_dict={X: X_batch, seq_length: len_batch, y: y_batch})
            mae_train = mae.eval(feed_dict={X: X_batch, seq_length: len_batch, y: y_batch})
            train_mae_list.append(float(mae_train))
            train_loss_list.append(loss_train)
            if batch_i % 5 == 0:
                print("{:4d}  Train loss: {:.4f}, Train MAE: {:.2f}  Val MAE: {:.2f}".format(
                epoch, loss_train, mae_train, mae_val))
        loss_hist['loss']['train'].append(float(np.array(train_loss_list).mean()))
        loss_hist['mae']['train'].append(float(np.array(train_mae_list).mean()))
        loss_val, step = sess.run(
                    [loss, global_step],
                    feed_dict={X: val_pad, seq_length: seq_lens_val, y: y_val})
        loss_hist['loss']['val'].append(loss_val)
        
        mae_val = mae.eval(feed_dict={X: val_pad, seq_length: seq_lens_val, y: y_val})
        loss_hist['mae']['val'].append(float(mae_val))
        
        
        best_ckpt_saver.handle(loss_val, sess, global_step)
            #saver.save(sess, "./models/model1.ckpt")
        #final_pred = y_pred.eval({X: val_pad, seq_length: seq_lens_val, y: y_test})

   0  Train loss: 17.0401, Train MAE: 1.88  Val MAE: 3.97
   0  Train loss: 30.4458, Train MAE: 3.13  Val MAE: 3.97
   0  Train loss: 18.9927, Train MAE: 2.45  Val MAE: 3.97
   0  Train loss: 50.7828, Train MAE: 3.10  Val MAE: 3.97
   0  Train loss: 55.7671, Train MAE: 3.70  Val MAE: 3.97
   1  Train loss: 41.1146, Train MAE: 3.27  Val MAE: 4.03
   1  Train loss: 41.2473, Train MAE: 2.93  Val MAE: 4.03
   1  Train loss: 11.1018, Train MAE: 2.82  Val MAE: 4.03
   1  Train loss: 8.5333, Train MAE: 2.16  Val MAE: 4.03
   1  Train loss: 39.9714, Train MAE: 3.27  Val MAE: 4.03
   2  Train loss: 29.2256, Train MAE: 3.19  Val MAE: 3.54
   2  Train loss: 27.4878, Train MAE: 3.37  Val MAE: 3.54
   2  Train loss: 14.0406, Train MAE: 2.57  Val MAE: 3.54
   2  Train loss: 39.7434, Train MAE: 3.02  Val MAE: 3.54
   2  Train loss: 22.0048, Train MAE: 2.86  Val MAE: 3.54
   3  Train loss: 15.0573, Train MAE: 2.55  Val MAE: 3.67
   3  Train loss: 58.9396, Train MAE: 2.88  Val MAE: 3.67
   3  Train los

In [ ]:
plt.figure(1)

#history of accuracy
plt.subplot(211)
plt.plot(loss_hist['loss']['train'])
plt.plot(loss_hist['loss']['val'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')  
plt.legend(['train', 'val'], loc='upper left')

In [ ]:
plt.figure(1)

#history of accuracy
plt.subplot(211)
plt.plot(loss_hist['mae']['train'])
plt.plot(loss_hist['mae']['val'])
plt.title('model loss')
plt.ylabel('mae')
plt.xlabel('epoch')  
plt.legend(['train', 'val'], loc='upper left')

## Test the model

In [59]:
saver = tf.train.Saver()
saver.restore(sess, checkmate.get_best_checkpoint(checkpoint_dir, select_maximum_value=False))

INFO:tensorflow:Restoring parameters from ./models/best.ckpt-5


RuntimeError: Attempted to use a closed Session.

In [ ]:
pred = y_pred.eval({X: test_pad, seq_length: test_lens, y: y_test})
np.abs((pred - y_test)).argmin()